In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [54]:
import torch
from torch import optim
import torchvision
from model.dataset import PneumoniaDataset
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

root = "C:\\Users\\Jason\\Downloads\\rsna-pneumonia-detection-challenge"

def collate_fn(batch):
    return tuple(zip(*batch))

In [63]:

model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes=2)
in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
hidden_layer = 256
model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                   hidden_layer,
                                                   num_classes=2)

In [ ]:
%%shell

# Using PyTorch helper functions to simplify training
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.3.0

In [71]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
dataset = PneumoniaDataset(root, True)
test_dataset = PneumoniaDataset(root, True)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
test_dataset = torch.utils.data.Subset(test_dataset, indices[-50:])

data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=2, shuffle=True, num_workers=0,
        collate_fn=collate_fn)

test_data_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1, shuffle=False, num_workers=0,
    collate_fn=collate_fn)

optimizer = optim.Adam(model.parameters(), lr=1e-6)
lr_step = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

# w = len(dataset)
# w
# data_loader = torch.utils.data.DataLoader(
#  dataset, batch_size=1, shuffle=True, num_workers=0)
#     # collate_fn=collate_fn)
# # For Training
# images,targets = next(iter(data_loader))
# images = list(image for image in images)
# idk = len(images)
# idk
# huh = len(targets)
# huh
# targets = [{k: v for k, v in t.items()} for t in [targets]]
# idk = len(targets)
# idk
# output = model(images,targets)   # Returns losses and detections
# # For inference
# model.eval()
# x = [torch.rand(1, 1024, 1024), torch.rand(1, 1024, 1024)]
# predictions = model(x)           # Returns predictions


RuntimeError: Input and output sizes should be greater than 0, but got input (D: 1, H: 2048, W: 2048) output (D: 0, H: 1600, W: 1600)